In [1]:
import cv2
import numpy as np

In [2]:
img = cv2.imread(r'C:\icbt\final_project\python\udamy_tests\data\aadhar_demo2.jpg')

# Loath the EAST Model

In [3]:
model = cv2.imread(r'C:\icbt\final_project\python\udamy_tests\models\frozen_east_text_detection.pb')

In [4]:
# Use multiple of 32 to set the new image shape
height, width, _ = img.shape
new_height = (height//32)*32
new_width = (width//32)*32
print(new_height, new_width)

384 640


In [5]:
# Get the ratio change in width and height
h_ratio = height / new_height
w_ratio = width / new_width
print(h_ratio, w_ratio)

1.0208333333333333 1.0
